# @2
# GHI CHÚ
# I) File này dùng để tạo mã khách hàng mới. Sau đó tạo và đưa dữ liệu vào bảng KHACH_HANG trong CSDL Ban_Hang
# II) Sử dụng dữ liệu sổ chi tiết bán hàng đã qua xử lý bằng file preprocess 

In [1]:
import pandas as pd
import numpy as np
import pyodbc

In [2]:
SCT20 = pd.read_csv('D://THUCTAP//data_da_xuly//SCT_20.csv', encoding='utf-16', sep='\t')
SCT21 = pd.read_csv('D://THUCTAP//data_da_xuly//SCT_21.csv', encoding='utf-16', sep='\t')
SCT22 = pd.read_csv('D://THUCTAP//data_da_xuly//SCT_22.csv', encoding='utf-16', sep='\t')

# B1 TẠO MÃ KH BAN ĐẦU

In [3]:
KHACH_HANG_20 = SCT20['Tên khách hàng'].unique()

In [4]:
KHACH_HANG_20 = pd.DataFrame({'Tên khách hàng': KHACH_HANG_20})

In [5]:
KHACH_HANG_20['Ma_KH'] = 'KH' + (KHACH_HANG_20.index + 1).astype(str).str.zfill(5)

In [6]:
KHACH_HANG_20.tail(5)

,Tên khách hàng,Ma_KH
110,Phan Thị Thuỷ,KH00111
111,Siêu thị Vimart,KH00112
112,CÔNG TY CP ĐẦU TƯ XÂY DỰNG VÀ THƯƠNG MẠI KHÁNH...,KH00113
113,CÔNG TY TNHH TƯ VẤN XÂY DỰNG DÂN DỤNG HÀ TĨNH,KH00114
114,Cty CP XD và TM Linh Châu,KH00115


In [7]:
def Unique_name(df):
    KH = df['Tên khách hàng'].unique()
    KH = pd.DataFrame({'Tên khách hàng': KH})
    return KH

# B2 Đưa dữ liệu vào CSDL

In [8]:
# Kết nối với CSDL
conn = pyodbc.connect(
Trusted_Connection = "Yes",
Driver = '{ODBC Driver 17 for SQL Server}',
Server = "DESKTOP-MDDIIDJ\MSSQLSERVER01",
Database = 'Ban_Hang')
cursor = conn.cursor()

In [9]:
#Tạo bảng KHACH_HANG
cursor.execute("CREATE TABLE KHACH_HANG(Ten_khach_hang nvarchar(150), Ma_KH varchar(8))")

In [10]:
#Thêm vào DS khách hàng ban đầu
def Insert_KH(df):
    for row in df.itertuples():
        cursor.execute('''
                        INSERT INTO Ban_Hang.dbo.KHACH_HANG(Ten_khach_hang, Ma_KH)
                        VALUES(?, ?)
                        ''',
                        row[1],
                        row[2]
                      )
    conn.commit()

In [11]:
# chuyển định dạng dữ liệu của bảng KH
def convert_columns_to_string(df):
    return df.astype(str)

In [12]:
KHACH_HANG_20=convert_columns_to_string(KHACH_HANG_20)

In [13]:
Insert_KH(KHACH_HANG_20)

In [14]:
# Tạo hàm thêm mới DS Khách hàng từ các năm tiếp theo.
def Insert_KH_n(df):
    for row in df.itertuples():
        # Kiểm tra xem tên khách hàng đã tồn tại trong cơ sở dữ liệu hay chưa
        cursor.execute('''
                        SELECT COUNT(*) 
                        FROM Ban_Hang.dbo.KHACH_HANG 
                        WHERE Ten_khach_hang = ?
                        ''',
                        row[1]
                      )
        count = cursor.fetchone()[0]
        
        if count == 0:  # Nếu tên khách hàng chưa tồn tại
            # Lấy ra ID lớn nhất hiện tại
            cursor.execute('''
                            SELECT MAX(CAST(SUBSTRING(Ma_KH, 3, LEN(Ma_KH)) AS INT)) 
                            FROM Ban_Hang.dbo.KHACH_HANG
                            ''')
            max_id = cursor.fetchone()[0]  # Lấy ID lớn nhất
            if max_id is None:
                new_id = 1
            else:
                new_id = max_id + 1
            
            # Tạo mã khách hàng mới
            new_kh_id = 'KH' + str(new_id).zfill(5)
            
            # Thêm dữ liệu mới vào cơ sở dữ liệu
            cursor.execute('''
                            INSERT INTO Ban_Hang.dbo.KHACH_HANG(Ten_khach_hang, Ma_KH)
                            VALUES(?, ?)
                            ''',
                            row[1],
                            new_kh_id
                          )
    conn.commit()

# KH năm 2021

In [15]:
# Lấy ra tên KH giao dịch trong năm 2021
KHACH_HANG_21 = Unique_name(SCT21)

In [16]:
# thay đổi định dạng DL cho DS KH năm 2021
KHACH_HANG_21 = convert_columns_to_string(KHACH_HANG_21)
# Thêm DS KH năm 2021 vào CSDL
Insert_KH_n(KHACH_HANG_21)

# KH năm 2022

In [17]:
# Lấy ra tên KH giao dịch trong năm 2022
KHACH_HANG_22 = Unique_name(SCT22)

In [18]:
# thay đổi định dạng DL cho DS KH năm 2022
KHACH_HANG_22 = convert_columns_to_string(KHACH_HANG_22)
# Thêm DS KH năm 2022 vào CSDL
Insert_KH_n(KHACH_HANG_22)